In [1]:
print("script initiated")
import time
very_start = time.time()
import argparse
import os
import sys
import tables as tb
sys.path.append("..")
cwd = os.getcwd()
from pathlib import Path
home = str(Path.home())
sys.path.append(home + "/ProdigyAI")
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
import h5py
import numpy as np
np.set_printoptions(suppress=True)  # don't use scientific notati
import pandas as pd
import pyarrow as pa

import pyarrow.parquet as pq
# from hanging_threads import start_monitoring

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler, StandardScaler
import keras
import third_party_libraries.finance_ml
import third_party_libraries.hudson_and_thames.mlfinlab as ml
import third_party_libraries.hudson_and_thames.mlfinlab.labeling.labeling as labeling
import third_party_libraries.hudson_and_thames.mlfinlab.sample_weights.attribution as attribution
import third_party_libraries.snippets as snp
from third_party_libraries.finance_ml.stats.vol import *

from library.core import *
# monitoring_thread = start_monitoring(seconds_frozen=360, test_interval=100)
# import googlecloudprofiler
# try:
#     googlecloudprofiler.start(
#         service="preemp-cpu-big-full-jeff_in-max",
#         # verbose is the logging level. 0-error, 1-warning, 2-info,
#         # 3-debug. It defaults to 0 (error) if not set.
#         verbose=3,
#     )
# except (ValueError, NotImplementedError) as exc:
#     print(exc)  # Handle errors here

arg_parse_stage = None
# Sorting out whether we are using the ipython kernel or not

try:
    get_ipython()
    check_if_ipython = True
    path_adjust = "../"
except Exception as e:
    check_if_ipython = False
    split_cwd = cwd.split("/")
    last_string = split_cwd.pop(-1)
    cwd = cwd.replace(last_string, "")
    os.chdir(cwd)
    parser = argparse.ArgumentParser(description="Preprocessing")
    parser.add_argument("-s",
                        "--stage",
                        type=str,
                        help="Stage of Preprocesssing")
    parser.add_argument("-m",
                        "--model",
                        type=str,
                        help="one_model or two_model")
    parser.add_argument("-f",
                        "--is_finished",
                        type=str,
                        help="Is this a continuation of preempted instance?")
    args = parser.parse_args()
    if args.stage != None:
        arg_parse_stage = 1
        if int(args.stage) == 1:
            if os.path.exists(path_adjust + "temp/data_name_gpu.txt"):
                os.remove(path_adjust + "temp/data_name_gpu.txt")
                print("removed temp/data_name_gpu.txt")
            else:
                print("The file does not exist")
    if args.model != None:
        model = args.model
    path_adjust = ""

if cwd == home + "/":
    cwd = cwd + "/ProdigyAI"
    path_adjust = cwd

try:
    with open(path_adjust + "temp/data_name_gpu.txt", "r") as text_file:
        gpu_file_name = text_file.read()
        stage = 2
except:
    stage = 1

side = None

with open(path_adjust + "temp/is_script_finished.txt", "w+") as text_file:
    text_file.write("start_script_time" + str(very_start))

if arg_parse_stage == 1:
    stage = int(args.stage)

print("the stage" + str(stage))
# Overide model and stage for testing

model = "two_model"
stage = 1
print("the overidden stage" + str(stage))

if stage == 2:
    # size
    h5f = h5py.File("data/gpu_output/" + gpu_file_name + ".h5", "r")
    X = h5f["X"][:]
    P = h5f["P"][:]
    sample_weights = h5f["sample_weights"][:]
    sampled_idx_epoch = h5f["sampled_idx_epoch"][:]
    h5f.close()
    data = pq.read_pandas("data/gpu_output/" + gpu_file_name +
                          "_data.parquet").to_pandas()
    X_for_all_labels = data.dropna(subset=["bins"])
    sampled_idx = pd.DatetimeIndex(sampled_idx_epoch)
    X_for_all_labels["predicted_bins"] = P
    side = X_for_all_labels["predicted_bins"]
    # Could use the probabilities (instead of [1,0,0] use [0.2,0.55,0.25]

yaml_path = path_adjust + "yaml/preprocessing.yaml"

with open(yaml_path) as file:
    yaml_dict = yaml.load(file, Loader=yaml.FullLoader)

config_dictionary = dict(yaml=yaml_path, params=yaml_dict)

(6, 0, 3)

In [2]:
import yaml
yaml_path = path_adjust + "yaml/preprocessing.yaml"

with open(yaml_path) as file:
    yaml_dict = yaml.load(file, Loader=yaml.FullLoader)

config_dictionary = dict(yaml=yaml_path, params=yaml_dict)

In [3]:
config_dictionary

{'yaml': '../yaml/preprocessing.yaml',
 'params': {'window_length': {'desc': 'Length of the input feature window',
   'value': 200},
  'profit_taking_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
   'value': 1},
  'stop_loss_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
   'value': 1},
  'minimum_return': {'desc': 'Amount of return chosen to consider it a profitable trade',
   'value': '0.001 * 1 / 23'}}}

In [4]:
config_dictionary.params.minimum_return

In [5]:
config_dictionary

{'yaml': '../yaml/preprocessing.yaml',
 'params': {'window_length': {'desc': 'Length of the input feature window',
   'value': 200},
  'profit_taking_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
   'value': 1},
  'stop_loss_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
   'value': 1},
  'minimum_return': {'desc': 'Amount of return chosen to consider it a profitable trade',
   'value': '0.001 * 1 / 23'}}}

In [6]:
config_dictionary['params']['minimum_return']

{'desc': 'Amount of return chosen to consider it a profitable trade',
 'value': '0.001 * 1 / 23'}

In [7]:
config_dictionary['params']['minimum_return'].value

In [8]:
wandb.init(
    dir="~/ProdigyAI/",
    project="prodigyai",
    config=config_dictionary,
)

In [9]:
import wandb

In [10]:
wandb.init(
    dir="~/ProdigyAI/",
    project="prodigyai",
    config=config_dictionary,
)

W&B Run: https://app.wandb.ai/garthtrickett/prodigyai/runs/2mak5z83

In [11]:
print('test')

In [12]:
wandb.config

In [13]:
wandb.config.minimum_return

In [14]:
dir(wandb.config)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_config_path',
 '_descriptions',
 '_items',
 '_load_defaults',
 '_load_file',
 '_load_values',
 '_load_wandb',
 '_run_dir',
 '_sanitize',
 '_sanitize_val',
 '_set_wandb',
 '_telemetry_update',
 '_update',
 '_wandb_dir',
 'as_dict',
 'desc',
 'from_environment_or_defaults',
 'get',
 'keys',
 'load_json',
 'persist',
 'set_run_dir',
 'setdefault',
 'setdefaults',
 'update',
 'user_items']

In [15]:
wandb.config.params

{'window_length': {'desc': 'Length of the input feature window', 'value': 200},
 'profit_taking_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
  'value': 1},
 'stop_loss_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
  'value': 1},
 'minimum_return': {'desc': 'Amount of return chosen to consider it a profitable trade',
  'value': '0.001 * 1 / 23'}}

In [16]:
wandb.config.params.minimum_return

In [17]:
wandb.config.params.minimum_return

In [18]:
wandb.config.params['minimum_return']

{'desc': 'Amount of return chosen to consider it a profitable trade',
 'value': '0.001 * 1 / 23'}

In [19]:
wandb.config.window_length

In [20]:
yaml_path = path_adjust + "yaml/tabl.yaml"

with open(yaml_path) as file:
    yaml_dict = yaml.load(file, Loader=yaml.FullLoader)

config_dictionary = dict(yaml=yaml_path, params=yaml_dict)

wandb.init(
    dir="~/ProdigyAI/",
    project="prodigyai",
    config=config_dictionary,
)

W&B Run: https://app.wandb.ai/garthtrickett/prodigyai/runs/1vud8k2s

In [21]:
wandb.config.batch_size

In [22]:
wandb.config.params

{'epochs': {'desc': 'Number of epochs to train over', 'value': 100},
 'batch_size': {'desc': 'Size of each mini-batch', 'value': 32}}

In [23]:
wandb.config.params.epochs

In [24]:
wandb.config.params['batch_size']['value']

32

In [25]:
config_dictionary['params']['minimum_return']['value']

In [26]:
config_dictionary['params']['minimum_return']['value']

In [27]:
wandb.config['params']

{'epochs': {'desc': 'Number of epochs to train over', 'value': 100},
 'batch_size': {'desc': 'Size of each mini-batch', 'value': 32}}

In [28]:
yaml_path = path_adjust + "yaml/preprocessing.yaml"

with open(yaml_path) as file:
    yaml_dict = yaml.load(file, Loader=yaml.FullLoader)

config_dictionary = dict(yaml=yaml_path, params=yaml_dict)

wandb.init(
    dir="~/ProdigyAI/",
    project="prodigyai",
    config=config_dictionary,
)

wandb.config['params']

{'window_length': {'desc': 'Length of the input feature window', 'value': 200},
 'profit_taking_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
  'value': 1},
 'stop_loss_multiplier': {'desc': 'Vertical Barrier profit taking multiplier',
  'value': 1},
 'minimum_return': {'desc': 'Amount of return chosen to consider it a profitable trade',
  'value': '0.001 * 1 / 23'}}